In [ ]:
############## URUCHOM TO NAJPIERW! ###################

# zaimportuj wszystko i zdefiniuj funkcję wykonującą testy
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [ ]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

In [ ]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

In [ ]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

In [ ]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

### Ćwiczenie 1

Napisz metodę `sig_hash` dla klasy `Tx`.

#### Wykonaj [ten test](/edit/r07/tx.py): `tx.py:TxTest:test_sig_hash`

In [ ]:
# Ćwiczenie 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

### Ćwiczenie 2

Napisz metodę `verify_input` dla klasy `Tx`. Użyj w tym celu metod `TxIn.script_pubkey`, `Tx.sig_hash` oraz `Script.evaluate`.

#### Wykonaj [ten test](/edit/r07/tx.py): `tx.py:TxTest:test_verify_p2pkh`

In [ ]:
# Ćwiczenie 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

In [ ]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

In [ ]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

In [ ]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

### Ćwiczenie 3

Napisz metodę `sign_input` dla klasy `Tx`.

#### Wykonaj [ten test](/edit/r07/tx.py): `tx.py:TxTest:test_sign_input`

In [ ]:
# Ćwiczenie 3

reload(tx)
run(tx.TxTest("test_sign_input"))

### Ćwiczenie 4

Utwórz transakcję testnetową wysyłającą 60% z jednego UTXO na adres `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. Pozostała kwota, pomniejszona o opłaty, powinna trafić na Twój adres przeznaczony na resztę. Powinna być to transakcja z jednym wejściem i z dwoma wyjściami.

Możesz rozesłać tę transakcję na https://live.blockcypher.com/btc/pushtx.

In [ ]:
# Ćwiczenie 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# utwórz 1 TxIn i 2 TxOut
# 1 z tych dwóch TxOut powinno trafić na Twój adres
# Drugie TxOut powinno trafić na ten adres:
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# Klucz prywatny weź z ćwiczenia w rozdziale 4.
# Adres dla reszty powinien być adresem wygenerowanym w rozdziale 4.

# prev_tx i prev_index weź z transakcji, w której dostałeś trochę monet testnetowych
# Utwórz wejście transakcyjne dla poprzedniej transakcji,
# używając domyślnego ScriptSig i nr porządkowego

# Kwota docelowa powinna być równa 60% kwoty wyjścia
# Wyznacz opłatę w rozsądnej kwocie
# Kwota reszty = kwota z poprzedniej tx - kwota docelowa - opłata

# Utwórz wyjście transakcyjne dla kwoty docelowej i adresu
# Utwórz wyjście transakcyjne dla kwoty reszty i adresu
# Utwórz obiekt transakcji

# Podpisz jedno wejście w obiekcie transakcji za pomocą klucza prywatnego
# Wydrukuj serializację transakcji szesnastkowo
# Roześlij, wysyłając do http://testnet.blockchain.info/pushtx

### Ćwiczenie 5

Dla ambitnych: zdobądź więcej monet testnetowych ze źródła testnetowego i utwórz transakcję z dwoma wejściami i z jednym wyjściem. Jedno wejście powinno pochodzić z takiego źródła, a drugie z poprzedniego ćwiczenia. Wyjście może być Twoim własnym adresem.

Możesz rozesłać tę transakcję na https://live.blockcypher.com/btc/pushtx.

In [ ]:
# Ćwiczenie 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Utwórz 2 TxIn, jedno z ćwiczenia nr 4 i jedno ze źródła testnetowego
# Utwórz 1 TxOut na powyższy adres
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# Klucz prywatny weź z ćwiczenia w rozdziale 4.

# prev_tx i prev_index weź z transakcji, w której dostałeś trochę monet testnetowych
# Utwórz pierwsze wejście transakcyjne, używając domyślnego ScriptSig i nr porządkowego
# prev_tx i prev_index weź z transakcji z ćwiczenia 4.
# Utwórz drugie wejście transakcyjne, używając domyślnego ScriptSig i nr porządkowego

# Wyznacz opłatę w rozsądnej kwocie
# Kwota docelowa powinna być sumą wejść - opłata

# Utwórz wyjście transakcyjne dla tej kwoty i dla adresu

# Podpisz pierwsze wejście za pomocą klucza prywatnego
# Podpisz drugie wejście za pomocą klucza prywatnego
# Wydrukuj serializację transakcji szesnastkowo
# Roześlij, wysyłając do http://testnet.blockchain.info/pushtx